# Lidar Remote Sensing of Snow

## Goal and Objectives

In this tutorial, we will learn how to work with lidar derived raster product that represent topography and snow depth. The objectives include:
1. Understanding the basic concept of lidar remote sensing
2. Deriving snow products from lidar DEMs differencing
3. Analyzing lidar raster data
4. Identifying available lidar data for snow science

The prerequisite for this tutorial is basic experience with python and geospatial data. 

## Overview

We will start by exploring point cloud data and take a quick look at how rasters are created from them. Then we will derive vegetation and snow depth by differencing elevation models followed by analysing these lidar data including DEM. Finally, we will address common issue with working with lidar data from varying sources. Finally, we will For the second part, we will talk about available snow focused lidar data.

## Introduction to Lidar Remote Sensing
Lidar stands for light detection and Ranging. It is a crucial remote sensing system for measuring elevation across wide areas. Airborne LiDAR technology is used to measure topography using a laser beam directed towards the ground with GPS and IMU systems providing the location and orientation of the airborne platform. 

<figure>
<img src = 'https://gmv.cast.uark.edu/wp-content/uploads/2013/01/ALS_scematic.jpg' style="width:100%">
<figcaption align =  'center'><b>Source: https://gmv.cast.uark.edu/scanning-2/airborne-laser-scanning//</b></figcaption>
</figure>


Depending on the feature the point return is coming from, elevation models can be Digital Terrain Model (DTM) or Digital Surface Model (DSM). DTM is the three-dimensional representation of the terrain without natural or man-made objects. DSM is a three-dimensional representation of the heights of the Earth's surface, including natural or man-made objects located on it.

<figure>
<img src = 'https://www.aw3d.jp/wp/wp-content/themes/AW3DEnglish/glossary/img/glo_01_01.jpg' style="width:100%">
<figcaption align =  'center'><b>Source: https://www.aw3d.jp/en/glossary/</b></figcaption>
</figure>

## Explore Point Cloud
In this activity, we will open a .las file, in the [plas.io free online lidar data viewer](http://plas.io/) and explore some attributes associated with a lidar data point cloud. 
1. Load the data: under file, select Autzen stadium. To navigate around, use left click and drag to tilt up and down, righ click and drag to move around. scroll bar to zoom in and out

2. Adjust Intesnity 

3. Change the lidar point cloud color options to classification

4. Explore


## Point to Pixels

Points clouds give us a lot of information (x, y, z and intensity). However, these data might be too large, overwhelming and difficult for scientific application. Lidar data are often provided in raster data format which is easier to work with in many GIS tools. Raster data can be generated from point clouds using gridding. 

[More...]


## Elevation Differencing

Canopy Height Model is a popular raster product from lidar data. It can be derived from differencing DSM and DEM. Snow depth is another raster product of interest. This is derived by differencing snow-on DEM and snow-off DEM. 
<figure>
<img src = 'https://i.imgur.com/uGkxpzQ.jpg' style="width:100%">
</figure>

We will difference the appropriate elevation model to derive vegetation height and snow depth. To begin, let's load the necessary packages for this tutorial.

In [ ]:
#import packages

import os

import numpy as np 

#plotting packages
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.xarray

#geospatial packages
import geopandas as gpd #for vector data
import xarray as xr
import rioxarray #for raster data
import rasterstats as rs


In [ ]:
#read the snow_on and snow_free DEM
cam_dem_snowOn = rioxarray.open_rasterio('./input/Cameron_Winter_DEM.tif', masked = True)
cam_dem_snowFree = rioxarray.open_rasterio('./input/Cameron_Summer_DEM.tif', masked =  True)

#derive snow depth
cam_snow_depth = cam_dem_snowOn - cam_dem_snowFree

#read the snow_free DSM to calculate vegetation height
cam_dsm_snowFree = rioxarray.open_rasterio('./input/Cameron_Summer_DSMs.tif', masked = True)

#derive vegetation height
cam_vh = cam_dsm_snowFree - cam_dem_snowFree


What does the DataArray object looks like?

In [ ]:
cam_snow_depth

The data has about 145 million values. Let's check the shape, CRS,extent and resolution and make a quick interactive plot.

In [ ]:
print('The shape of the data is is:', cam_snow_depth.shape)
print('\nCRS of the data is:', cam_snow_depth.rio.crs)
print('\n The spatial extent of the data is:', cam_snow_depth.rio.bounds())
print('\n The resolution of the data is:', cam_snow_depth.rio.resolution())


Let's quickly see the plot and see the distribution of pixel values

In [ ]:
cam_snow_depth.hvplot.image(x= 'x', y = 'y', rasterize = True, cmap = 'viridis', height = 500)

Let's subset over small area for faster plotting and further analysis.

In [ ]:
cam_dem_roi = cam_dem_snowFree.sel(y = slice(4489168, 4487655), x = slice(425077, 425848))
cam_vh_roi = cam_vh.sel(y = slice(4489168, 4487655), x = slice(425077, 425848))
cam_sd_roi = cam_snow_depth.sel(y = slice(4489168, 4487655), x = slice(425077, 425848))

In [ ]:
# Set font size and font family
plt.rcParams.update({'font.size': 18})
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

#create fig and axes elements
fig, axs = plt.subplots(ncols = 3, figsize=(18, 12))

#plot the data
cam_dem_roi.plot(ax = axs[0], cmap = 'Greys', robust = True, cbar_kwargs={'label': 'Elevation (m)', 'orientation':'horizontal'}) 
cam_vh_roi.plot(ax = axs[1], cmap = 'summer_r', robust = True, cbar_kwargs={'label': 'Vegetation Height (m)', 'orientation':'horizontal'})
cam_sd_roi.plot(ax = axs[2], cmap = 'Blues', robust = True, cbar_kwargs={'label': 'Snow Depth (m)', 'orientation':'horizontal'})

#Set the title
# fig.suptitle('Cameron Study Area', fontsize = 20)

#set the axes title and tick locators
axs[0].set_title('')
axs[0].xaxis.set_major_locator(plt.LinearLocator(3))
axs[1].set_title('')
axs[1].xaxis.set_major_locator(plt.LinearLocator(3))
axs[2].set_title('')
axs[2].xaxis.set_major_locator(plt.LinearLocator(3))

plt.tight_layout()
plt.show()

## More Analysis

Lidar provides highly accurate DEM, vegetation height and snow depth. The purpose of this section is to quickly demonstrate some of the analysis we can do with these products. We will sample snow depth, elevation and vegetation height at 100 locations to quantify snow distribution over the the subset region

In [ ]:
#import point location data
cam_100_points = gpd.read_file('./input/cameron_100_points.gpkg')

#create a buffer around the point feature and save to file
cam_100_points.buffer(20).to_file('./Results/cam_100_points_buffer.gpkg')

Let's see the sampled points

In [ ]:
cam_100_poly = gpd.read_file('/home/naheemadebisi/snow-analytics/SnowEx_Lidar_Tutorial/Results/cam_100_points_buffer.gpkg')
fig, ax = plt.subplots(figsize=(10,12))
cam_sd_roi.plot(ax = ax)
cam_100_poly.plot(ax = ax, color = 'black')
plt.show()

In [ ]:
#Extract raster values for the vegetation height
cam_VH_stat = rs.zonal_stats('./Results/cam_100_points_buffer.gpkg', 
            cam_vh_roi.squeeze().values,
            nodata = -9999,
            geojson_out=True,
            affine = cam_vh_roi.rio.transform(), 
            stats = 'count mean')
cam_VH_stat_df =  gpd.GeoDataFrame.from_features(cam_VH_stat) #turn the geojson to a dataframe
cam_VH_stat_df.rename(columns = {'mean': 'VH_mean', 'count': 'VH_count'}, inplace = True) #rename the columns

#Extract raster values for the snow depth
cam_SD_stat = rs.zonal_stats(cam_VH_stat_df,
            cam_sd_roi.squeeze().values,
            nodata = -9999,
            geojson_out=True,
            affine = cam_sd_roi.rio.transform(),
            stats = 'count mean')
cam_SD_stat_df =  gpd.GeoDataFrame.from_features(cam_SD_stat) #turn the geojson to a dataframe
cam_SD_stat_df.rename(columns = {'mean': 'SD_mean', 'count': 'SD_count'}, inplace = True) #rename the columns

#Extract raster values for elevation
cam_DEM_stat = rs.zonal_stats(cam_SD_stat_df,
            cam_dem_roi.squeeze().values,
            nodata = -9999,
            geojson_out=True,
            affine = cam_dem_roi.rio.transform(),
            stats = 'count mean')
cam_DEM_stat_df =  gpd.GeoDataFrame.from_features(cam_DEM_stat) #turn the geojson to a dataframe
cam_DEM_stat_df.rename(columns = {'mean': 'ELEV_mean', 'count': 'ELEV_count'}, inplace = True) #rename the columns

#rename the dataframe
vh_sd_dem = cam_DEM_stat_df


In [ ]:
# Set font size and font family
plt.rcParams.update({'font.size': 18})
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

#create figure and gridspec object
fig = plt.figure(figsize=(20,12), constrained_layout=True)
gspec = fig.add_gridspec(ncols=5, nrows=5)

ax0 = fig.add_subplot(gspec[0, 0])
ax1 = fig.add_subplot(gspec[0, 1])
ax2 = fig.add_subplot(gspec[0, 2])
ax3 = fig.add_subplot(gspec[1:5, 0])
ax4 = fig.add_subplot(gspec[1:5, 1])
ax5 = fig.add_subplot(gspec[1:5, 2])
ax6 = fig.add_subplot(gspec[0:5, 3])
ax7 = fig.add_subplot(gspec[0:5, 4])

# plot the boxplot of elevation, vegetation and snow thickness
sns.boxplot(x= vh_sd_dem['ELEV_mean'], ax=ax0, palette= 'colorblind')
ax0.set_xlabel('')
sns.boxplot(x= vh_sd_dem['VH_mean'], ax=ax1, palette= 'colorblind')
ax1.set_xlabel('')
sns.boxplot(x= vh_sd_dem['SD_mean'], ax=ax2, palette= 'colorblind')
ax2.set_xlabel('')

# plot the histogram and boxplot of elevation, vegetation and snow thickness
sns.histplot(data = vh_sd_dem, x ='ELEV_mean', ax=ax3, bins=50)
ax3.set_xlabel('Elevation (m)')
sns.histplot(data = vh_sd_dem, x ='VH_mean', ax=ax4, bins=50, binrange=(1,10))
ax4.set_xlabel('Vegetation Height (m)')
sns.histplot(data = vh_sd_dem, x ='SD_mean', ax=ax5, bins=50, binrange=(0,1.5))
ax5.set_xlabel('Snow Depth (m)')

#make the scatter plot of elevation vs snow thickness and vegetation vs snow thickness
sns.kdeplot(data = vh_sd_dem, x = 'SD_mean', y = 'ELEV_mean', shade=True, ax=ax6, cmap='Blues')
ax6.set_xlabel('Snow Depth (m)')
ax6.set_ylabel('Elevation (m)')
#set xlimt and y limt
#ax6.set_xlim(0, 2.5)
ax6.set_ylim(2900, 3300)

sns.kdeplot(data = vh_sd_dem, x = 'SD_mean', y = 'VH_mean', shade=True, ax=ax7, cmap='Blues')
ax7.set_xlabel('Snow Depth (m)')
ax7.set_ylabel('Vegetation Height (m)')
#set xlimt and y limt
#ax7.set_xlim(0, 2.5)
ax7.set_ylim(-3, 15)

plt.show()

## Common Pitfall in Elevation differencing

The lidar data we have used so far are acquired from the same source (QSI). This made the DEM differencing to be quiet straighforward because the crs, resolution and extent are the same. At times, the elevation data might be from diffeerence sources. It will thus require extra effort of resampling and reprojection to accurately derive snow depth. 
Let's do DEM differencing again but using snow-on and snow-off DEM from different sources. The snow-on data is an 0.5m resolution raster acquired by  QSI over GrandMesa in 2020. A 3m DEM data acquired by ASO in 2016 will be used as the snow-off DEM. 

As a first step, we have to downsample the QSI data to the resolution of the ASO.

In [ ]:
!gdal_translate  -tr 3, 3 ./input/QSI_0.5m_GrandMesa13Feb_Winter_DEM.tif ./Results/QSI_3m_GrandMesa13Feb_Winter_DEM.tif

In [ ]:
#check the raster metadata
!gdalinfo ./Results/QSI_3m_GrandMesa13Feb_Winter_DEM.tif

In [ ]:
#read the data as arrays
snow_on_dem = rioxarray.open_rasterio('./Results/QSI_3m_GrandMesa13Feb_Winter_DEM.tif', masked =True)
snow_on_dem.hvplot.image(x= 'x', y = 'y', rasterize = True, cmap = 'viridis')

In [ ]:
#read the snow-off DEMs
snow_free_dem = rioxarray.open_rasterio('./input/ASO_3M_GrandMesa_Summer_DEM.tif',
            masked=True)
snow_free_dem.hvplot.image(x= 'x', y = 'y', rasterize = True, cmap = 'viridis')

The extent and crs of the snow-off DEM is different from the snow-on. Let's match the snow-off DEM with the snow-on DEM

In [ ]:
snow_free_dem_match = snow_free_dem.rio.reproject_match(snow_on_dem)
snow_free_dem_match.hvplot.image(x= 'x', y = 'y', rasterize = True, cmap = 'viridis')

In [ ]:
snow_depth = snow_on_dem - snow_free_dem_match
snow_depth.hvplot.image(x= 'x', y = 'y', rasterize = True, cmap = 'viridis')

About 17 m snow depth.... What's the issue?

We need to do vertical datum transformation. Matching the vertical datum of data from different sources is important for elevation data. 
The vertical datum of ASO is in ellipsoid (WGS 84)while QSI is in geoid (NAVD). Let's transform ASO to same datum as QSI. 

For the QSI, projection:

UTM Zone 12 North

Horizontal Datum: NAD83 (2011). NAD 83 is a 3-D reference system, it also serves as the horizontal control datum for the United States, Canada, Greenland and Central America using the Geodetic Reference System 1980 (GRS 80) ellipsoid, an earth-centered reference ellipsoid. [See this link for more reading](https://vdatum.noaa.gov/docs/datums.html)

Vertical Datum: NAVD88 (Geoid12b)
Units: Meters

<figure>
<img src = 'https://i.imgur.com/CbvMuMB.jpg' style="width:100%">
<figcaption align =  'center'><b>Relationship between Clarke 1866, WGS 84, GRS 80 and Geoid (left).Relationship between elliposid, Geoid and Orthometric Height (Right). Source: NOAA</b></figcaption>
</figure>





In [ ]:
!gdalsrsinfo -o proj4 ./input/ASO_3M_GrandMesa_Summer_DEM.tif

In [ ]:
!gdalsrsinfo -o proj4 ./Results/QSI_3m_GrandMesa13Feb_Winter_DEM.tif

In [ ]:
#coordinate transformation
!gdalwarp -s_srs "+proj=utm +zone=13 +datum=WGS84 +units=m +no_defs" -t_srs "+proj=utm +zone=12 +ellps=GRS80 +units=m +no_defs +geoidgrids=egm96_15.gtx" ./input/ASO_3M_GrandMesa_Summer_DEM.tif ./Results/ASO_3M_GrandMesa_Summer_DEM_geoid.tif

In [ ]:
#read the snow-off DEMs
snow_free_dem = rioxarray.open_rasterio('./Results/ASO_3M_GrandMesa_Summer_DEM_geoid.tif',
            masked=True)
            
#reproject the ASO DEM to match the QSI DEM
snow_free_dem_match = snow_free_dem.rio.reproject_match(snow_on_dem)

In [ ]:
snow_depth = snow_on_dem - snow_free_dem_match
snow_depth = snow_depth.where((snow_depth > 0) & (snow_depth < 3))
snow_depth.hvplot.image(x= 'x', y = 'y', rasterize = True, cmap = 'viridis')

## Available Data for Lidar Remote Sensing

Various lidar data has been acquired to monitory snow over the western US.
1. 2013 - 2019: ASO data
2. 2020 Quantunmn data over some selected areas.
3. 2021 Quantumn data over selected areas
4. 2022: Riegl flight over MCS

For the 2020 and 2021 data we have lidar data including 
The lidar data we have include:

1. DEMs (SnowOn and SnowFree)
2. DSMs (SnowOn and SnowFree)
3. Vegetation Height (SnowOn and SnowFree)
4. Snow Depth
5. Intensity Images (SnowOn and SnowFree)

![](https://i.imgur.com/Y5gsOFO.jpg)

References:
1. Earthdatascience